# ATIAM FpA computer science

## Assignment

This script defines the overall exercise for ATIAM structure course

 - Use this as a baseline script
 - You are authorized to define other files for functions
 - Write a (small) report document (PDF) explaining your approach
 - All your files should be packed in a zip file named
     [ATIAM][FpA2020]FirstName_LastName.zip

@author: esling

In [1]:
%load_ext autoreload
%autoreload 2
from typing import Any, Callable, Union
import importlib
import util as ut
# importlib.reload(ut)

# Basic set of imports (here you can see if everything passes)
from music21 import converter
import numpy as np
import math
from needleman import needleman_affine
from needleman import needleman_simple
import string
import pickle
import os
import pretty_midi as midi
import re
import matplotlib.pyplot as plt

rng = np.random.default_rng()

('CEELECANTH', '-PELICAN--', 13)


In [2]:
unit_testing = True
unit_testing_log_level = 2


midi_database = pickle.load(open("atiam-fpa.pkl", "rb"))
composers = midi_database['composers']
composers_tracks = midi_database['composers_tracks']

# Here an example: print all composers with more than 10 tracks
for composer, tracks in sorted(composers_tracks.items()):
    if (len(tracks) >= 10):
        print(f"{composer} :  {len(tracks)} tracks")

Abel, Carl Friedrich :  27 tracks
Aboyan, Gayk :  554 tracks
Abt, Franz :  38 tracks
Adam, Adolphe :  11 tracks
Adson, John :  71 tracks
Agincour, François d' :  15 tracks
Agrell, Johan :  84 tracks
Agricola, Alexander :  12 tracks
Aguado, Dionisio :  28 tracks
Ahle, Johann Rudolf :  18 tracks
Aiblinger, Johann Kaspar :  18 tracks
Aichinger, Gregor :  12 tracks
Alain, Jehan :  13 tracks
Albeniz, Isaac :  49 tracks
Albert, Heinrich :  10 tracks
Albinoni, Tomaso :  170 tracks
Albrechtsberger, Johann Georg :  89 tracks
Aleotti, Vittoria :  18 tracks
Alexandra, Liana :  12 tracks
Alink, Bert :  22 tracks
Alkan, Charles-Valentin :  30 tracks
Allegri, Lorenzo :  12 tracks
Allison, Richard :  177 tracks
Alsen, Wulf Dieter :  38 tracks
Altenburg, Johann Ernst :  28 tracks
Altenburg, Michael :  26 tracks
Alıcıoğlu, Şafak :  75 tracks
Ammer, Manfred :  114 tracks
Anderson, Leigh :  10 tracks
André, Johann Anton :  11 tracks
Anglebert :  67 tracks
Anonymous :  27 tracks
Anonymus, .... :  899 

### PART 1 - Exploring a track collections (text dictionnaries) and playing with MIDI

In this part, we will start easy by looking at a collection of tracks.
The set of classical music pieces is provided in the _atiam-fpa.pkl_ file, which
is already loaded at this point of the script and contain two structures
    - composers         = Array of all composers in the database
    - composers_tracks  = Hashtable of tracks for a given composer


### Q-1.2 Use your own algorithm to sort the collection of composers by decreasing number of tracks

In [27]:
## Some unit testing
use_threads = False
if unit_testing:
    for i in range(100):
        dim = rng.integers(1, 4)
        shape=tuple(rng.integers(1, 20, size=dim))
        n = np.multiply.reduce(shape)
        axis=rng.integers(0, dim)
        if unit_testing_log_level <= 1:
            print(f"shape={shape}, axis={axis}")
        arr = rng.integers(0, n, size=n)
        rng.shuffle(arr)
        arr = arr.reshape(shape)
        arr_sorted_truth = np.sort(arr, axis=axis)
        #
        arr_sorted = ut.pipsorted(arr, axis=axis, use_threads=use_threads)
        #
        if unit_testing_log_level <= 2:
            assert(np.linalg.norm(arr_sorted-arr_sorted_truth)<0.001)

In [ ]:
# see: https://stackoverflow.com/q/15579649
# and: https://stackoverflow.com/a/43187340
composers_tracks_tuple = [(comp, composers_tracks[comp] if comp in composers_tracks else []) for comp in composers]

composers_tracks_tuple_sorted = ut.pipsorted(composers_tracks_tuple, order=lambda a, b: len(a[1])-len(b[1]))[::-1]
composers_tracks_sorted = dict(composers_tracks_tuple_sorted.copy())
for composer, tracks in composers_tracks_sorted.items():
    if len(tracks) > 0:
        print(f"Composer: {composer}, # of tracks={len(tracks)}")

### Q-1.3 Extend your sorting procedure, to sort all tracks from all composers alphabetically

In [ ]:
def lexi_comp(a: str, b: str) -> int:
    # print(a, b)
    i = 0
    while i < min(len(a), len(b)):
        if a[i] > b[i]:
            return 1
        elif a[i] < b[i]:
            return -1
        i += 1
    if len(a) > len(b):
        return 1
    elif len(a) < len(b):
        return -1
    return 0

tracks_sorted = [None] * len(composers_tracks_tuple_sorted)
for i in range(len(composers_tracks_tuple_sorted)):
    tracks_sorted[i] = ut.pipsorted(composers_tracks_tuple_sorted[i][1], order=lambda a, b: lexi_comp(a, b))
composers_tracks_sorted_2 = dict([(composers_tracks_tuple_sorted[i][0], tracks_sorted[i]) for i in range(len(tracks_sorted))])
for composer, tracks in composers_tracks_sorted_2.items():
   if len(tracks) > 0:
       print(f"Composer: {composer}, #={len(tracks)} -- {tracks[::len(tracks)//5+1]}")

## MIDI part 

### In addition to the pickle file, you can find some example MIDI files in the atiam-fpa/ folder.

### Here we are going to import and plot the different MIDI files. We recommend using the pretty_midi library

    pip install pretty_midi

### But you can rely on any method (even code your own if you want)


### Q-1.4 Import and plot some MIDI files

### Based on the provided MIDI files (random subset of Beethoven tracks), try to import, plot and compare different files.

In [ ]:
# midi_dir = './atiam-fpa/'
# pat = 'beethoven_[0-9]*.[mid|midi|MID|MIDI]'
midi_dir = './partitions/'
pat = r'[a-zA-Z_]*\.[mid|midi|MID|MIDI]'

files = [os.path.abspath(os.path.join(midi_dir, x)) for x in os.listdir(midi_dir) if re.match(pat, x)]
get_part_name = lambda f: os.path.splitext(os.path.basename(f))[0]
parts = np.array([(get_part_name(file), midi.PrettyMIDI(file)) for file in files], dtype=[('name', 'U20'), ('midi', midi.PrettyMIDI)])
get_key_name = np.vectorize(lambda i: midi.key_number_to_key_name(i))
get_note_name = np.vectorize(lambda i: midi.note_number_to_name(i))

fig = plt.figure('Pitch class histrogrammes', figsize=(8, 5))
axis = fig.add_subplot()
axis.legend(parts['name'])
for (i, part) in enumerate(parts):
    notes = get_note_name(np.arange(12))
    axis.plot(notes, part['midi'].get_pitch_class_histogram()) 

### Q-1.5 Compute the number of notes in a MIDI and sort the collection

### First write a function counting the number of notes played in a given MIDI file.
### Then, sort the set of MIDI files based on the number of notes.

In [ ]:
def count_notes(m: midi.PrettyMIDI):
    # see: https://stackoverflow.com/a/42889025
    larr = np.array([len(l) for l in ut.get_instrument_notes(m).values()])
    return np.sum(larr, axis=None)

parts_sorted = ut.pipsorted(parts, axis=0, order=lambda p1, p2: count_notes(p1['midi']) - count_notes(p2['midi']))[::-1]
for part in parts_sorted:
    print(f" Song: {part['name']}, # of notes: {count_notes(part['midi'])}")


### PART 2 - Symbolic alignments and simple text dictionaries

In this part, we will use our knowledge on computer structures to solve a very
well-known problem of string alignment. Hence, this part is split between:

1) Implement a string alignment
2) Try to apply this to a collection of classical music pieces names
3) Develop your own more adapted procedure to have a matching inside large set

### Question 1 - Reimplementing the simple NW alignment

### Q-2.1 Here perform your Needleman-Wunsch (NW) implementation.

- You can find the definition of the basic NW [here](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm)
    - In this first version, we will be implementing the _basic_ gap costs
    - Remember to rely on a user-defined matrix for symbols distance


In [4]:
def format_name(name: str) -> str:
    # we only care about letters, we remove spaces, numbers and special characters.
    return re.compile("[^a-zA-Z]").sub('', name).casefold().upper()

f_1 = './atiam-fpa_alpha.dist'
f_2 = './atiam-fpa_dna.dist'
sim_mat, letters = ut.load_similarity_matrix(f_1)

if unit_testing:
    for i in range(50):
        gap_penality=rng.integers(-6, -1)
        l_1, l_2 = rng.integers(4, 12, size=(2,))
        s_1, s_2 = format_name(ut.get_random_word(l_1)),format_name(ut.get_random_word(l_2))
        aligned_2 = needleman_simple(s_1, s_2,  matrix=f_1, gap=gap_penality)
        aligned = ut.pipman(s_1, s_2, sim_mat=sim_mat, letters=letters, gap_penality=gap_penality)
        # print(aligned)
        if unit_testing_log_level <= 2 and aligned_2[2] != aligned['score']:
            print('Results for basic gap costs (linear)')
            print(f"Score={aligned_2[2]}")
            print(f"-: {aligned_2[0]} // {aligned_2[1]}")
            print('Pip -- Results for basic gap costs (linear)')
            print(f"Score={aligned['score']}")
            print("Paths:")
            for k in range(len(aligned['string_list'])):
                print(f"{k}: {aligned['string_list'][k][0]} // {aligned['string_list'][k][1]}")

Results for basic gap costs (linear)
Score=-30
-: KSQZFDYT // --LZZLCC
Pip -- Results for basic gap costs (linear)
Score=-22
Paths:
0: KSQZFDYT // --LZZLCC
1: KSQZFDYT // -L-ZZLCC
2: KSQZFDYT // L--ZZLCC
3: KSQZFDYT // -LZZ-LCC
4: KSQZFDYT // L-ZZ-LCC
5: KSQZFDYT // LZ-Z-LCC
6: KSQZFDYT // -LZZL-CC
7: KSQZFDYT // L-ZZL-CC
8: KSQZFDYT // LZ-ZL-CC
9: KSQZFDYT // -LZZLC-C
10: KSQZFDYT // L-ZZLC-C
11: KSQZFDYT // LZ-ZLC-C
12: KSQZFDYT // -LZZLCC-
13: KSQZFDYT // L-ZZLCC-
14: KSQZFDYT // LZ-ZLCC-
Results for basic gap costs (linear)
Score=-31
-: -KW-----PXZ // HQWFTGAETZV
Pip -- Results for basic gap costs (linear)
Score=-23
Paths:
0: -KW----PXZ- // HQWFTGAETZV
1: K-W----PXZ- // HQWFTGAETZV
2: -KW---P-XZ- // HQWFTGAETZV
3: K-W---P-XZ- // HQWFTGAETZV
4: -KW--P--XZ- // HQWFTGAETZV
5: K-W--P--XZ- // HQWFTGAETZV
6: -KW-P---XZ- // HQWFTGAETZV
7: K-W-P---XZ- // HQWFTGAETZV
8: -KWP----XZ- // HQWFTGAETZV
9: K-WP----XZ- // HQWFTGAETZV
10: -KW---PX-Z- // HQWFTGAETZV
11: K-W---PX-Z- // HQWFTGAETZV
12:

### Q-3.2 Extending the NW algorithm
    
- Add the affine gap penalty to your original NW algorithm
- You can use the Gotoh algorithm reference
- Verify your code by using the provided compiled version

In [5]:
gap_penality=[-5, -2]
s_1 = "CEELECANTH"
s_2 = "PELICAN"

if unit_testing:
    for i in range(50):
        gap_penality=ut.pipsorted(rng.integers(-6, -1, size=(2,)))
        l_1, l_2 = rng.integers(4, 12, size=(2,))
        s_1, s_2 = format_name(ut.get_random_word(l_1)),format_name(ut.get_random_word(l_2))
        aligned_2 = needleman_affine(s_1, s_2,  matrix=f_1, gap_open=gap_penality[0], gap_extend=gap_penality[1])
        aligned = ut.pipman_affine(s_1, s_2, sim_mat=sim_mat, letters=letters, gap_penality=gap_penality)
        # print(aligned)
        if unit_testing_log_level <= 2 and aligned_2[2] != aligned['score']:
            print('Results for affine gap costs')
            print(f"Score={aligned_2[2]}")
            print(f"-: {aligned_2[0]} // {aligned_2[1]}")
            # print(aligned)
            print('Pip -- Results for affine gap costs')
            print(f"Score={aligned['score']}")
            print("Paths:")
            for k in range(len(aligned['string_list'])):
                print(f"{k}: {aligned['string_list'][k][0]} // {aligned['string_list'][k][1]}")

Results for affine gap costs
Score=-29
-: ZRKTPRBCQN // M-UZVGZDEA
Pip -- Results for affine gap costs
Score=-25
Paths:
0: --ZRKTPRBCQN // MUZ---VGZDEA
1: --ZRKTPRBCQN // MUZ--V-GZDEA
2: --ZRKTPRBCQN // MUZ-V--GZDEA
3: --ZRKTPRBCQN // MUZV---GZDEA
4: --ZRKTPRBCQN // MUZ--VG-ZDEA
5: --ZRKTPRBCQN // MUZ-V-G-ZDEA
6: --ZRKTPRBCQN // MUZV--G-ZDEA
7: --ZRKTPRBCQN // MUZ-VG--ZDEA
8: --ZRKTPRBCQN // MUZV-G--ZDEA
9: --ZRKTPRBCQN // MUZVG---ZDEA
10: --ZRKTPRBCQN // MUZ--VGZ-DEA
11: --ZRKTPRBCQN // MUZ-V-GZ-DEA
12: --ZRKTPRBCQN // MUZV--GZ-DEA
13: --ZRKTPRBCQN // MUZ-VG-Z-DEA
14: --ZRKTPRBCQN // MUZV-G-Z-DEA
15: --ZRKTPRBCQN // MUZVG--Z-DEA
16: --ZRKTPRBCQN // MUZ-VGZ--DEA
17: --ZRKTPRBCQN // MUZV-GZ--DEA
18: --ZRKTPRBCQN // MUZVG-Z--DEA
19: --ZRKTPRBCQN // MUZVGZ---DEA
20: --ZRKTPRBCQN // MUZ--VGZD-EA
21: --ZRKTPRBCQN // MUZ-V-GZD-EA
22: --ZRKTPRBCQN // MUZV--GZD-EA
23: --ZRKTPRBCQN // MUZ-VG-ZD-EA
24: --ZRKTPRBCQN // MUZV-G-ZD-EA
25: --ZRKTPRBCQN // MUZVG--ZD-EA
26: --ZRKTPRBCQN // MUZ-VGZ-D-EA

### Question 2 - Applying this to a collection of musical scores


The set of classical music pieces is provided in the *atiam-fpa.pkl* file, which
is already loaded at this point of the script and contain two structures
- `composers`         = Array of all composers in the database
- `composers_tracks`  = Hashtable of tracks for a given composer

Some examples of the content of these structures

    composers[23] => 'Abela, Placido'
    composers[1210]  => 'Beethoven, Ludwig van'

    composers_tracks['Abela, Placido'] => ['Ave Maria(Meditation on Prelude No. 1 by J.S.Bach)']
    composers_tracks['Beethoven, Ludwig van'] => ['"Ode to Joy"  (Arrang.)', '10 National Airs with Variations, Op.107 ', ...]

    composers_tracks['Beethoven, Ludwig van'][0] => '"Ode to Joy"  (Arrang.)'

### Q-2.2 Apply the NW algorithm between all tracks of each composer

- For each track of a composer, compare to all remaining tracks of the same composer
- Establish a cut criterion (what is the relevant similarity level ?) to only print relevant matches
- Propose a set of matching tracks and save it through Pickle

In [ ]:

def comp(a, b):
    s_1 = a[1]['score']
    s_2 = b[1]['score']
    return s_1 - s_2
composer_1 = 'Bohm, Carl'
composer_2 = 'Harrington, Jeffrey Michael'

file_name = 'bohm_aligned.pkl'

track_names = composers_tracks[composer_1]
track_alignment_its = dict()
# We will first sort by score, then display the best.
for (i, track_name) in enumerate(track_names):
    other_track_names = track_names[:i] + track_names[i+1:]
    alignments = [ut.pipman(format_name(track_name), format_name(other_track_name), sim_mat=sim_mat, letters=letters, gap_penality=gap_penality) for other_track_name in other_track_names]
    track_alignment = dict(zip(other_track_names, alignments))
    track_alignment_its[track_name]= ut.pipsorted(list(track_alignment.items()), order=lambda a,b: a[1]['score']-b[1]['score'])[::-1]
    # We will first sort by score, then keep the best ones for each track.
# Get the best score for this composer
alignment_scores = np.array([[track_alignment_its[track_name][i][1]['score'] for i in range(len(track_names)-1)] for track_name in track_names])
best_score = np.max(alignment_scores)
# our threshold will depend on this best score.
score_threshold = 0.8 * best_score
best_alignments = []
for track_name in track_names:
    for (other_track_name, alignment) in track_alignment_its[track_name]:
        if alignment['score'] > score_threshold:
            print(f"{track_name} // {other_track_name}: {alignment['score']} --> {alignment['string_list'][0][0]} // {alignment['string_list'][0][1]}")
            best_alignments.append(alignment)
with open(file_name, 'wb') as f:
    pickle.dump(best_alignments, f)

### Q-2.3 Extend your previous code so that it can compare

- A given track to all tracks of all composers (full database)
- You should see that the time taken is intractable (computational explosion)
- Propose a method to avoid such a huge amount of computation
- Establish a cut criterion (what is relevant similarity)
- Propose a set of matching tracks and save it through Pickle

#### We could use the [method of the 'Four Russians'](https://en.wikipedia.org/wiki/Method_of_Four_Russians) to reduce complexity


## PART 3 - Extending the alignment algorithm and musical matching

You might have seen from the previous results that
- Purely string matching on classical music names is not the best approach
- This mostly comes from the fact that the importance of symbols is not the same
- For instance:
    - "Symphony for orchestra in D minor"
    - "Symphony for orchestra in E minor"
    Looks extremely close but the key is the most important symbol

Another flaw in our approach is that the NW algorithm treats all gaps
equivalently. Hence, it can put lots of small gaps everywhere.

Regarding alignement, it would be better to have long coherent gaps rather
than small ones. 

This is handled by a mecanism known as _affine gap penalty_
which separates the costs of either _opening_ or _extending_ a gap.

This is known as the Gotoh algorithm, which can be 
found [here](http://helios.mi.parisdescartes.fr/~lomn/Cours/BI/Material2019/gap-penalty-gotoh.pdf).


### Q-3.1 Extending to a true musical name matching

### Start by exploring the collection for well-known composers, what do you see ?

- Propose a new name matching algorithm adapted to classical music piece names
- These are only given as indicative ideas ...
    - Can be based on a rule-based system
    - Can be a pre-processing for symbol finding and then an adapted weight matrix
    - Can be a local-alignement procedure
- Implement this new comparison procedure adapted to classical music piece names
- Re-run your previous results (Q-2.2 and Q-2.3) with this procedure

In [ ]:
# TODO
a = -np.inf
b = np.inf
a + b